In [91]:
# 百度api 
ak_01 = 'FknGql9MZgddnZnSeeI7RQ7R'
sk_01 = 'ZavTBlEt8kXcsUguUXMl6IHW0SkGi3pw'

ak_02 = 'O1TNgyHXyhoElxqGYjm9Wc07'
sk_02 = 'YvB8ILTCOASAoWRmAl3lh8KiEy0YcDVB'

In [92]:
import requests
import json

# 获取 token
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={}&client_secret={}'.format(ak_02,sk_02)

res = requests.post(host)
print(res.text[:400])
access_token = json.loads(res.text)
token = access_token['access_token']
token

{"refresh_token":"25.19afcfef12ba9652daba7c3f94e59992.315360000.1900134866.282335-18973636","expires_in":2592000,"session_key":"9mzdCKYjwDmvnb345DmPlvN3CAWPCt8ybf7Re5GlqZBTh+IxDjJpkejlxRYEBcqRgkRzpdpzNpuEMy5imxYQlL4v2op0zQ==","access_token":"24.3b2e17cc37a8b5820a3bc47c450f7a1c.2592000.1587366866.282335-18973636","scope":"public nlp_simnet nlp_wordemb nlp_comtag nlp_dnnlm_cn brain_nlp_lexer brain_a


'24.3b2e17cc37a8b5820a3bc47c450f7a1c.2592000.1587366866.282335-18973636'

In [83]:
def getTextSentiment(keyOrder, text, token):
    '''
    todo：返回对应文本的情感分类结果 
        keyOrder：该条文本的标识数字
        text：文本
        token：百度云开放平台口令
    '''
    url = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token={}'.format(token)
    data = {
        'text':str(text)
           }
    data = json.dumps(data)
    response = requests.post(url,data=data)
    response.keep_alive = False # 关闭多余连接
    return response.text

token = '24.f3ec606dbf1a97967b0cf7ab4dcb1b81.2592000.1587287055.282335-18942611'
text = '加油敬汉卿'

result = getTextSentiment(1,text, token)
result

'{"log_id": 2234426013324592309, "text": "加油敬汉卿", "items": [{"positive_prob": 0.992705, "confidence": 0.98379, "negative_prob": 0.00729473, "sentiment": 2}]}'

In [69]:
def precessRespAndInsert(result):
    '''
    todo:解析结果，插入数据库中
    '''
    score = []
    log_id = json.loads(result)['log_id']  # 调用api会出现错误，需要做个返回结果检测
    if(log_id != "" ):
        items = json.loads(result)['items']
        item_confidence = items[0]['confidence']
        item_negative_prob = items[0]['negative_prob']
        item_positive_prob = items[0]['positive_prob']
        score.append(item_confidence)
        score.append(item_positive_prob)
        score.append(item_negative_prob)
        return score
        #print("该文本自信度：{}，积极词性得分：{}，消极词性得分：{}。".format(item_confidence, item_positive_prob, item_negative_prob))

        

precessRespAndInsert(result)

[0.998463, 0.0006915, 0.999309]

In [55]:
import pymysql

def getText(keyOrder):
    # 打开数据库连接
    db = pymysql.connect(host = "localhost", port = 3306, user = "root", password = "123456", database = 'processSentiment', charset='utf8')
    # 使用cursor()方法获取操作游标
    cursor = db.cursor()
    # SQL 插入语句
    sql = 'SELECT message FROM BiliComment_NoRepeat WHERE keyOrder = '+str(keyOrder)+';'
    #print(sql)
    try:
        # 执行sql语句
        #print("prefer execute sql.  ")
        cursor.execute(sql)
        # 提交到数据库执行
        db.commit()
        result = cursor.fetchall()[0][0]
        #print(result)
        # 关闭数据库连接
        db.close()
        return result
    except:
        print("worng in insert")
        # 如果发生错误则回滚
        db.rollback()
        db.close()
        
r =  getText(2)
r

'好早'

In [65]:
def updateData(text_condence, text_pos_score, text_neg_score, keyOrder):
    # 打开数据库连接
    db = pymysql.connect(host = "localhost", port = 3306, user = "root", password = "123456", database = 'processSentiment', charset='utf8')

    # 使用cursor()方法获取操作游标
    cursor = db.cursor()
    # SQL 插入语句
    sql = "UPDATE BiliComment_NoRepeat "\
          "SET text_confidence ='"+str(text_condence)+"', text_pos_score='"+str(text_pos_score)+"' ,text_neg_score = '"+str(text_neg_score)+"' "\
          "WHERE keyOrder='"+str(keyOrder)+"';"
    #print(sql)
    try:
        # 执行sql语句
        #print("prefer execute sql.  ")
        cursor.execute(sql)
        # 提交到数据库执行
        db.commit()
        # 关闭数据库连接
        db.close()
    except:
        print("worng in insert")
        # 如果发生错误则回滚
        db.rollback()
        db.close()
        
updateData(1.121234, 2.3452222, 3.0987654, 2)

In [95]:
import random

def main(keyOrder, token_list):
    token = random.choice(token_list)
    text = getText(keyOrder)
    result = getTextSentiment(keyOrder, text, token)
    scoce_list = precessRespAndInsert(result)
    updateData(scoce_list[0], scoce_list[1], scoce_list[2], keyOrder)
    print("has run:"+str(keyOrder))
    time.sleep(0.5)

In [96]:
import time
#text_number = 20
#text_number = 132339
#strat_number = 1681
token_01 = '24.f3ec606dbf1a97967b0cf7ab4dcb1b81.2592000.1587287055.282335-18942611'
token_02 = '24.3b2e17cc37a8b5820a3bc47c450f7a1c.2592000.1587366866.282335-18973636'
token_list = [token_01,token_02]

for keyOrder in range(strat_number, text_number+1):
    try:
        main(keyOrder, token_list)
    except:
        time.sleep(5)
        print("wrong here:"+str(keyOrder))
        main(keyOrder, token_list)

    


has run:1681
has run:1682
has run:1683
has run:1684
has run:1685
has run:1686
has run:1687
has run:1688
has run:1689
has run:1690
has run:1691
has run:1692
has run:1693
has run:1694
has run:1695
has run:1696
has run:1697
has run:1698
has run:1699
has run:1700
has run:1701
has run:1702
has run:1703
has run:1704
has run:1705
has run:1706
has run:1707
has run:1708
has run:1709
has run:1710
has run:1711
has run:1712
has run:1713
has run:1714
has run:1715
has run:1716
has run:1717
has run:1718
has run:1719
has run:1720
has run:1721
has run:1722
has run:1723
has run:1724
has run:1725
has run:1726
has run:1727
has run:1728
has run:1729
has run:1730
has run:1731
has run:1732
has run:1733
has run:1734
has run:1735
has run:1736
has run:1737
has run:1738
wrong here:1739
has run:1739
has run:1740
has run:1741
has run:1742
has run:1743
has run:1744
has run:1745
has run:1746
has run:1747
has run:1748
has run:1749
has run:1750
has run:1751
has run:1752
has run:1753
has run:1754
has run:1755
has run:1

KeyboardInterrupt: 

55s 110-150 即  40条55s
总计 133000 

50小时 我可去


换 pycharm了